<a href="https://colab.research.google.com/github/maxmatical/pytorch-projects/blob/master/xresnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#XResnet

Implementation of xresnet from https://arxiv.org/abs/1812.01187 and https://github.com/fastai/fastai/blob/master/fastai/vision/models/xresnet.py


In [0]:
import torch.nn as nn

In [0]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
        
    def forward(self, x):
        return x.view(x.size(0), -1)
    
# weight initialization
def init_cnn(m):
    if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
    if isinstance(m, (nn.Conv2d,nn.Linear)): nn.init.kaiming_normal_(m.weight)
    for l in m.children(): init_cnn(l)

# activation function
act_fn = nn.ReLU(inplace=True)
        

In [0]:
# default conv layer
def conv(in_channels, out_channels, kernel_size = 3, stride = 1, bias = False):
    return nn.Conv2d(in_channels, out_channels, kernel_size = kernel_size, stride = stride, padding = kernel_size//2, bias = bias)


# conv + bn + act_fun
def conv_layer(in_channels, out_channels, kernel_size=3, stride=1, zero_bn=False, act=True):
    bn = nn.BatchNorm2d(out_channels)
    nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
    layers = [conv(in_channels, out_channels, kernel_size, stride = stride), bn]
    if act: 
        layers.append(act_fn)
    return nn.Sequential(*layers)
       



In the Resblock, the last conv_layer is set to zero_bn 

This is because if zero_bn is true, the entire conv_layer is set to 0 since the bn sets the outputs to 0

This lets the NN learn when a resblock is not needed, it learns to skip over the block if it doesn't contribute

If it contributes, then it learns the weight of the bn so then the resblock actually has weights

In [0]:
# residual block

def no_op(x): return x # no operations done if in_channels == out_channels


class Resblock(nn.Module):
    def __init__(self, expansion, in_channels, mid_channels, stride = 1):
        super(Resblock, self).__init__()
        out_channels, in_channels = mid_channels*expansion, in_channels*expansion
        if expansion == 1:
            layers = [conv_layer(in_channels, mid_channels, 3, stride = stride), #either stride 1 (if res block) or stride = 2 for downsampling blocks
                      conv_layer(mid_channels, out_channels, 3, zero_bn = True, act = False)] # stride 1
        else:
            layers = [conv_layer(in_channels, mid_channels, 1),
                      conv_layer(mid_channels, mid_channels, 3, stride = stride), #either stride 1 (if res block) or stride = 2 for downsampling blocks
                      conv_layer(mid_channels, out_channels, 1, zero_bn = True, act = False)]
            
        self.convs = nn.Sequential(*layers)
        
        # if in_channels != out_channels, use a 1x1 conv to get the same channels, otherwise return x (no operations)
        self.idconv = no_op if in_channels == out_channels else conv_layer(in_channels, out_channels, 1, act = False)
        
        self.pooling = no_op if stride == 1 else nn.AvgPool2d(2, ceil_mode=True)
        
    def forward(self, x):
        x1 = self.convs(x) # convs operations
        x2 = self.idconv(self.pooling(x)) # pooling and 1x1 conv layer operations
        out = x1+x2
        return act_fn(out)


For XResNet. Replaces the 7x7 stride 2 stem with 3 3x3 convs (stride = 2 for first conv) in a row

This is done beecause 3x3 convolutions are much less computationally expensive than a 7x7 (5.4 times more expensive)

In [0]:
# creating XResNet
class XResNet(nn.Sequential):
    def __init__(self, expansion, layers, in_channels = 3, n_classes=1000):
        
        """
        layers = list of length 4. 
        layer[i] = how many resblocks in each of the 4 chunks of the network
        expansion = what value to multiply the intermediate n_out of the convlayer by
        """
        #stem
        stem = []
        stem_sizes = [in_channels, 32, 32, 64]
        for i in range(3):
            stem.append(conv_layer(stem_sizes[i], stem_sizes[i+1], stride = 2 if i==0 else 1))
            
        # creating the resblock layers
        block_sizes = [64//expansion, 64, 128, 256, 512]
        blocks = [self._make_layer(expansion, in_channels = block_sizes[i], out_channels = block_sizes[i+1], blocks = l, stride = 1 if i == 0 else 2) #1st stage has no downsampling
                    for i, l in enumerate(layers)] #l in enumerate(layers) goes through layers in XResNet and sets the value of blocks based on layer[i]
        
        # creating network
        super().__init__(*stem,
                      nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                      *blocks,
                      nn.AdaptiveAvgPool2d(1), 
                      Flatten(),
                      nn.Linear(block_sizes[-1]*expansion, n_classes)
        )
        
        init_weight(self)
        
    def _make_layer(self, expansion, in_channels, out_channels, blocks, stride):
        """
        blocks = int. -> number of blocks to create = layer[i]
        """
        return nn.Sequential(
            *[Resblock(expansion, in_channels if i == 0 else out_channels, out_channels, stride if i==0 else 1) # only stride 2 for the downsampling block(first block) and stride = 1 for residual blocks
              for i in range(blocks)])


In [0]:
xresnet_18 = XResNet(expansion = 1, layers = [2, 2, 2, 2])
xresnet_34 = XResNet(expansion = 1, layers = [3, 4, 6, 3])
xresnet_50 = XResNet(expansion = 4, layers = [3, 4, 6, 3])
xresnet_101 = XResNet(expansion = 4, layers = [3,4,23,3])
xresnet_152 = XResNet(expansion = 4, layers = [3,8,36,3])
